In [1]:
from dpp_nets.layers.layers import *
import torch
import torch.nn as nn
from collections import OrderedDict
import shutil
import time
import gzip
import os
import json
import numpy as np
from dpp_nets.utils.io import make_embd, make_tensor_dataset, load_tensor_dataset
from dpp_nets.utils.io import data_iterator, load_embd
from torch.autograd import Variable
from torch.utils.data.dataloader import DataLoader
import time
from dpp_nets.my_torch.utilities import pad_tensor

In [2]:
## Data Sets
train_set = torch.load('/Users/Max/data/beer_reviews/pytorch/annotated_common.pt')
rat_set = torch.load('/Users/Max/data/beer_reviews/pytorch/annotated.pt')
embd = load_embd('/Users/Max/data/beer_reviews/pytorch/embeddings.pt')

In [3]:
# Parameters
batch_size = 25
_, max_set_size = train_set.data_tensor.size()
_, embd_dim = embd.weight.size()

hidden_dim = 500
enc_dim = 200
target_dim = 3 # let's choose the first three aspects to learn!

# Baseline
baseline_nets = DeepSetBaseline(embd_dim, hidden_dim, enc_dim, target_dim)
baseline = nn.Sequential(embd, baseline_nets, nn.Sigmoid())

# Model
kernel_dim = 200
kernel_net = KernelVar(embd_dim, hidden_dim, kernel_dim)
sampler = MarginalSampler()
pred_net = PredNet(embd_dim, hidden_dim, enc_dim, target_dim)
trainer = MarginalTrainer(kernel_net, sampler, pred_net)

trainer.reg = 0.1
trainer.reg_mean = 10
trainer.activation = nn.Sigmoid()

train_loader = DataLoader(train_set, batch_size, shuffle=True)

In [4]:
# Actual training loop for model

params = [{'params': trainer.kernel_net.parameters(), 'lr': 1e-3},
          {'params': trainer.pred_net.parameters(), 'lr': 1e-4}]

optimizer = torch.optim.Adam(params)
trainer.reg = 0.1

for epoch in range(10):
    for t, (review, target) in enumerate(train_loader):
        words = embd(Variable(review))
        target = Variable(target[:,:3])
        loss  = Rtrainer(words, target)
        
        # Backpropagate + parameter updates
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if not (t+1) % 10: 
            print('Loss at it :', t+1, 'is', loss.data[0])
            
            


NameError: name 'Rtrainer' is not defined

In [ ]:
# Need also a training script for RTrainer!!
sampler = ReinforceSampler(5)
Rtrainer = ReinforceTrainer(trainer.kernel_net, sampler, trainer.pred_net)

In [ ]:
# Actual training loop for baseline
# Training
criterion = nn.MSELoss()
lr = 1e-4
optimizer = torch.optim.Adam(baseline_nets.parameters(), lr=lr)


for epoch in range(10):
    
    for t, (review, target) in enumerate(train_loader):
        target = Variable(target[:,:3])
        words = Variable(review)
        pred = baseline(words)
        loss = criterion(pred, target)

        # Backpropagate + parameter updates
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if not (t+1) % 10: 
            print('Loss at it :', t+1, 'is', loss.data[0])

In [ ]:
def validate_baseline(val_set, model, criterion):
    x = Variable(val_set.data_tensor, volatile=True)
    y = Variable(val_set.target_tensor[:,:3], volatile=True)
    pred = model(x)
    loss = criterion(pred, y)
    print(loss.data[0])

In [ ]:
def validate_model(val_set, model):
    model.reg = 0
    x = Variable(val_set.data_tensor, volatile=True)
    x = embd(x)
    y = Variable(val_set.target_tensor[:,:3], volatile=True)
    loss = model(x, y)
    print(loss.data[0])

In [ ]:
validate_model(train_set, trainer)

In [ ]:
validate_baseline(train_set, baseline, nn.MSELoss())

In [ ]:
import random
def sample(model, sampler, embd, dataset):
    rand = random.randint(0, len(dataset))
    x = dataset.data_tensor[rand:rand+2]
    x = embd(Variable(x))
    y = dataset.target_tensor[rand:rand+2]
    kernel = trainer.kernel_net(x)
    sampler.s_ix = trainer.kernel_net.s_ix
    sampler.e_ix = trainer.kernel_net.e_ix
    sampler(kernel, x)
    print(sampler.saved_subsets)

In [ ]:
rand = random.randint(0, len(train_set))
x = train_set.data_tensor[rand:rand+10]
x = embd(Variable(x))
y = Variable(train_set.target_tensor[rand:rand+10,:3])
Rtrainer(x, y)

In [ ]:
[i.data.sum() for l in Rtrainer.sampler.saved_subsets for i in l]

In [8]:
A = torch.randn(9,3)

In [9]:
A


-1.3224 -0.2710 -1.0688
 0.5904  0.7979  0.7390
-0.0869  2.1480 -2.9223
-1.1593 -0.3784 -0.4341
 0.1043 -1.1186  0.7613
-0.5970 -1.4440 -2.8684
-0.5452 -0.7874 -0.7970
-0.8491 -0.9683  0.6335
-0.7317 -0.0121  0.5305
[torch.FloatTensor of size 9x3]

In [10]:
A.view(3,3,3)


(0 ,.,.) = 
 -1.3224 -0.2710 -1.0688
  0.5904  0.7979  0.7390
 -0.0869  2.1480 -2.9223

(1 ,.,.) = 
 -1.1593 -0.3784 -0.4341
  0.1043 -1.1186  0.7613
 -0.5970 -1.4440 -2.8684

(2 ,.,.) = 
 -0.5452 -0.7874 -0.7970
 -0.8491 -0.9683  0.6335
 -0.7317 -0.0121  0.5305
[torch.FloatTensor of size 3x3x3]

In [31]:
batch_size = 3
alpha_iter = 3
target_dim = 1
target = torch.randn(batch_size * alpha_iter)
pred = torch.randn(batch_size * alpha_iter)

In [36]:
losses = Variable((target - pred).view(batch_size, alpha_iter,target_dim).sum(2))

In [37]:
[[i.data[0] for i in row] for row in losses]

[[2.39558482170105, 0.42840495705604553, 0.8365748524665833],
 [-0.24185743927955627, 0.6139535903930664, 0.8181594610214233],
 [0.3858824372291565, -1.586434245109558, -0.04904495179653168]]

In [38]:
losses


Variable containing:
(0 ,.,.) = 
  2.3956
  0.4284
  0.8366

(1 ,.,.) = 
 -0.2419
  0.6140
  0.8182

(2 ,.,.) = 
  0.3859
 -1.5864
 -0.0490
[torch.FloatTensor of size 3x3x1]